In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from langsmith import Client

client = Client()
dataset = client.clone_public_dataset(
    "https://smith.langchain.com/public/89ef0d44-a252-4011-8bb8-6a114afc1522/d"
)

In [16]:
from openai import OpenAI
openai_client = OpenAI()
from pydantic import BaseModel, Field

class Toxicity(BaseModel):
    toxicity: str = Field(description="""'Toxic' if this the statement is toxic, 'Not toxic' if the statement is not toxic.""")

def good_classifier(inputs: dict) -> dict:
    completion = openai_client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": f"This is the statement: {inputs['statement']}"
            }
        ],
        response_format=Toxicity,
    )

    toxicity_score = completion.choices[0].message.parsed.toxicity
    return {"class": toxicity_score}

In [17]:
def f1_score_summary_evaluator(outputs: list[dict], reference_outputs: list[dict]) -> dict:
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for output_dict, reference_output_dict in zip(outputs, reference_outputs):
        output = output_dict["class"]
        reference_output = reference_output_dict["class"]
        if output == "Toxic" and reference_output == "Toxic":
            true_positives += 1
        elif output == "Toxic" and reference_output == "Not toxic":
            false_positives += 1
        elif output == "Not toxic" and reference_output == "Toxic":
            false_negatives += 1

    if true_positives == 0:
        return {"key": "f1_score", "score": 0.0}

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)
    return {"key": "f1_score", "score": f1_score}

In [18]:
results = client.evaluate(
    good_classifier,
    data=dataset,
    summary_evaluators=[f1_score_summary_evaluator],
    experiment_prefix="Good classifier"
)

View the evaluation results for experiment: 'Good classifier-9b87f82e' at:
https://smith.langchain.com/o/bcaa99f9-3a04-43e7-9c9d-c034ad1da1ea/datasets/b6e63993-ced7-458c-be33-86197f220c89/compare?selectedSessions=3fc74a03-cce7-4476-b596-c0db5915a4e5




9it [00:14,  1.66s/it]
